## GRU

In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf

sess = tf.Session()
import numpy as np

# 用tensorflow 导入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/MNIST_data', one_hot=True) 

# 看看咱们样本的数量
print(mnist.test.labels.shape)
print(mnist.train.labels.shape)

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
(10000, 10)
(55000, 10)


In [2]:
lr = 1e-3
input_size = 28
timestep_size = 28
hidden_size = 256
layer_num = 2
class_num = 10
cell_type = 'block_gru'

X_input = tf.placeholder(tf.float32, [None, 784])
y_input = tf.placeholder(tf.float32, [None, class_num])

batch_size = tf.placeholder(tf.int32, [])
keep_prob = tf.placeholder(tf.float32, [])

#### 1.搭建gru模型

In [3]:
#将784维字符信息转化为28*28的图片
#1.RNN的输入shape(batch_size, timestep_size, input_size)
X = tf.reshape(X_input, [-1, 28, 28])

#2.创建gru结构
def gru_cell(cell_type, num_nodes, keep_prob):
    assert(cell_type in ["gru", "block_gru"], "wrong cell type.")
    if cell_type == "gru":
        cell = tf.contrib.rnn.GRUCell(num_nodes)
    else:
        cell = tf.contrib.rnn.GRUBlockCell(num_nodes)
    cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell
    
mgru_cell = tf.contrib.rnn.MultiRNNCell([gru_cell(cell_type, hidden_size, keep_prob) for _ in range(layer_num)], state_is_tuple=True)
#3.初始化state
init_state = mgru_cell.zero_state(batch_size, dtype = tf.float32)

#4.方法一调用dynamic_rnn()来让构建好的网络运行
#outputs, state = tf.nn.dynamic_rnn(mgru_cell, inputs=X, initial_state=init_state, time_major=False)
#h_state = state[-1][1]

#4.方法二按照时间步展开
outputs = list()
state = init_state
with tf.variable_scope('RNN'):
    for timestep in range(timestep_size):
        (cell_output, state) = mgru_cell(X[:, timestep, :], state)
        outputs.append(cell_output)
h_state = outputs[-1]

In [4]:
import time

#开始训练&测试
W = tf.Variable(tf.truncated_normal([hidden_size, class_num], stddev=0.1), dtype=tf.float32)
bias = tf.Variable(tf.constant(0.1, shape=[class_num]), dtype=tf.float32)
y_pre = tf.nn.softmax(tf.matmul(h_state, W) + bias)

#损失和评估函数
cross_entropy = -tf.reduce_mean(y_input * tf.log(y_pre))
train_op = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(y_input, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())

time0 = time.time()
for i in range(5000):
    _batch_size=100
    X_batch, y_batch = mnist.train.next_batch(batch_size=_batch_size)
    cost, acc,  _ = sess.run([cross_entropy, accuracy, train_op], feed_dict={X_input: X_batch, y_input: y_batch, keep_prob: 0.5, batch_size: _batch_size})
    if (i+1) % 500 == 0:
        # 分 100 个batch 迭代
        test_acc = 0.0
        test_cost = 0.0
        N = 100
        for j in range(N):
            X_batch, y_batch = mnist.test.next_batch(batch_size=_batch_size)
            _cost, _acc = sess.run([cross_entropy, accuracy], feed_dict={X_input: X_batch, y_input: y_batch, keep_prob: 1.0, batch_size: _batch_size})
            test_acc += _acc
            test_cost += _cost
        print("step {}, train cost={:.6f}, acc={:.6f}; test cost={:.6f}, acc={:.6f}; pass {}s".format(i+1, cost, acc, test_cost/N, test_acc/N, time.time() - time0))
        time0 = time.time()

step 500, train cost=0.010583, acc=0.970000; test cost=0.011339, acc=0.964400; pass 90.70239162445068s
step 1000, train cost=0.011326, acc=0.970000; test cost=0.006237, acc=0.981000; pass 90.98501682281494s
step 1500, train cost=0.002188, acc=1.000000; test cost=0.006315, acc=0.981300; pass 90.85975360870361s
step 2000, train cost=0.003981, acc=0.980000; test cost=0.005769, acc=0.982500; pass 90.84781837463379s
step 2500, train cost=0.011619, acc=0.990000; test cost=0.004633, acc=0.986400; pass 90.8639965057373s
step 3000, train cost=0.016436, acc=0.970000; test cost=0.004883, acc=0.985000; pass 91.1623969078064s
step 3500, train cost=0.011160, acc=0.980000; test cost=0.004238, acc=0.988200; pass 90.98647427558899s
step 4000, train cost=0.000364, acc=1.000000; test cost=0.003797, acc=0.988900; pass 90.88716650009155s
step 4500, train cost=0.006923, acc=0.990000; test cost=0.003516, acc=0.990000; pass 90.94028973579407s
step 5000, train cost=0.000362, acc=1.000000; test cost=0.003804, a